In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline 
%matplotlib notebook
import sklearn
import skimage
from sklearn import svm, metrics, datasets
from sklearn.utils import Bunch
from sklearn.model_selection import GridSearchCV, train_test_split

import os
from skimage.transform import resize
import pylab


from scipy import ndimage, misc

import cv2

In [ ]:
# เปิดโฟลเดอร์จาก google drive นะ
main_folder_path = "/content/drive/MyDrive/Ai/Dataset/"
save_folder_path = "/content/drive/MyDrive/Ai/Dataset1/"
main_folder = os.listdir(main_folder_path)
save_folder = os.listdir(save_folder_path)

# Preprocess

In [ ]:
i = 0
j = 0
for sub_folder in main_folder:
    print(sub_folder)
    # gluacoma, normal, other
    list_folder = os.listdir(main_folder_path+sub_folder)
    count_f = 0
    for image_folder in list_folder:
      print(image_folder)
      # subfolder in gluacoma , normal , others
      list_image = os.listdir(main_folder_path+sub_folder+'/'+image_folder)
      j = int(len(list_image)/5) if len(list_image) % 5 == 0 else int(len(list_image)/5) + 1
      print(list_folder,len(list_image), j )
      i = 0
      count = 0
      # count_f = 0
      for k, image in enumerate(list_image) :
        # Contour
        im = cv2.imread(main_folder_path+sub_folder+'/'+image_folder+'/'+image)
        im_rgb = cv2.cvtColor(im,cv2.COLOR_BGR2RGB)
        w_real, h_real, _ = np.array(im_rgb).shape

        # Contour -> หาเส้นรอบวง
        im_gr = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY) # แปลงเป็นขาวดำ
        _, im_thr = cv2.threshold(im_gr,10,255,0)
        # หาเส้นเค้าโครง
        contour, _ = cv2.findContours(im_thr,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
        max_contour = np.argmax([len(i) for i in contour])
        con = im_rgb
        x,y,w,h = cv2.boundingRect(contour[max_contour])
        # print(w,h)
        if w >= h :
          w_cat = w
          h_plus = w-h
          con = con[y:y+w, x:x+w]
          if w > w_real:
            img_plus = np.zeros([int(h_plus/2),w_cat,3],dtype=np.uint8)
            img_plus.fill(0)
            con = np.concatenate((img_plus[:None],con,img_plus[:None]),axis = 0) #0 short edge, 1 long edge
        else :
          w_cat = h-w
          h_plus = h
          con = con[y:y+h, x:x+h]
        # Resized
        im_resized = cv2.resize(con, (224,224))
        if count < j-1 and k < len(list_image) - 1:
          cv2.imwrite(f"/content/drive/MyDrive/Ai/Dataset1/{sub_folder}/{i}/{image[:-4]}.png", cv2.cvtColor(im_resized, cv2.COLOR_BGR2RGB))
          count += 1
          count_f += 1
        else :
          cv2.imwrite(f"/content/drive/MyDrive/Ai/Dataset1/{sub_folder}/{i}/{image[:-4]}.png", cv2.cvtColor(im_resized, cv2.COLOR_BGR2RGB))
          count += 1
          count_f += 1
          i += 1
          print(count, k+1)
          count = 0
      print(count_f)

Normal
ORIGA
['ORIGA', 'UniOfErlangen-Nuremberg', 'Kaggle', 'Orange-IEEE-Dataport'] 482 97
97 97
97 194
97 291
97 388
94 482
482
UniOfErlangen-Nuremberg
['ORIGA', 'UniOfErlangen-Nuremberg', 'Kaggle', 'Orange-IEEE-Dataport'] 15 3
3 3
3 6
3 9
3 12
3 15
497
Kaggle
['ORIGA', 'UniOfErlangen-Nuremberg', 'Kaggle', 'Orange-IEEE-Dataport'] 38 8
8 8
8 16
8 24
8 32
6 38
535
Orange-IEEE-Dataport
['ORIGA', 'UniOfErlangen-Nuremberg', 'Kaggle', 'Orange-IEEE-Dataport'] 188 38
38 38
38 76
38 114
38 152
36 188
723
Gluacoma
Technische_Fakultat
['Technische_Fakultat', 'BinRushed2', 'BinRushed3', 'BinRushed4', 'MagrabiaFemale', 'MagrabiaMale', 'BinRushed1', 'ORIGA', 'MessidorPrime'] 15 3
3 3
3 6
3 9
3 12
3 15
15
BinRushed2
['Technische_Fakultat', 'BinRushed2', 'BinRushed3', 'BinRushed4', 'MagrabiaFemale', 'MagrabiaMale', 'BinRushed1', 'ORIGA', 'MessidorPrime'] 47 10
10 10
10 20
10 30
10 40
7 47
62
BinRushed3
['Technische_Fakultat', 'BinRushed2', 'BinRushed3', 'BinRushed4', 'MagrabiaFemale', 'MagrabiaMale',

In [ ]:
for sub_folder in save_folder:
    # print(sub_folder)
    # gluacoma, normal, other
    list_folder = os.listdir(save_folder_path+'/'+sub_folder)
    for image_folder in list_folder:
      # print(image_folder)
      # subfolder in gluacoma , normal , others
      list_image = os.listdir(save_folder_path+sub_folder+'/'+image_folder)
      print(f"{sub_folder}/{image_folder}",len(list_image))

In [ ]:
glucoma_folder = os.listdir("/content/drive/MyDrive/Ai/Testset/Gluacoma/")
print(len(glucoma_folder))

94


In [ ]:
def clahe(img_in):
  def clahe(img_in):
  #img_in = cv2.cvtColor(img_in, cv2.COLOR_BGR2GRAY)
  equ = cv.equalizeHist(img_in)
  res = np.hstack((img_in,equ))
  im_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
  im_clahe.apply(equ)
  plt.imshow(res)
  return im_clahe


In [ ]:
%matplotlib inline 
for i, img in enumerate(glucoma_folder):
  if img == "Gluacoma_1427.png":
    im = cv2.imread(f"/content/drive/MyDrive/Ai/Testset/Gluacoma/{img}")
    im_rgb = cv2.cvtColor(im,cv2.COLOR_BGR2RGB)
    im_rgb = np.array(im_rgb)
    im_gry = cv2.cvtColor(im_rgb,cv2.COLOR_RGB2GRAY)
    im_gry = np.array(im_gry)

    # Feature
    im_clahe = clahe(im_gry)

    im_gr = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY) # แปลงเป็นขาวดำ
    _, im_thr = cv2.threshold(im_gr,10,255,0)
    # หาเส้นเค้าโครง
    contour, _ = cv2.findContours(im_thr,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
    print(len(contour))

    max_contour = np.argmax([len(i) for i in contour])
    con_t = cv2.drawContours(im_rgb,contour,max_contour, (0,200,0), 5)

    x,y,w,h = cv2.boundingRect(contour[max_contour])
    print(w,h)
    if w >= h :
      w_cat = w
      h_plus = w-h
      con = con_t[y:y+w, x:x+w]
      if w > w_real:
        img_plus = np.zeros([int(h_plus/2),w_cat,3],dtype=np.uint8)
        img_plus.fill(0)
        print(con.shape, img_plus.shape)
        con = np.concatenate((img_plus[:None],con,img_plus[:None]),axis = 0) #0 short edge, 1 long edge
    else :
      w_cat = h-w
      h_plus = h
      con = con_t[y:y+h, x:x+h]


    im_resized = cv2.resize(con, (224,224))
    plt.imshow(im_gry)

